This Notebook connects to a mongodb database, connects to the Lakeshore temperature monitors that we are using, creates a data object by scanning these instruments, and inserts them into the MongoDB database.

In [1]:
import time
import numpy as np
from datetime import datetime
from calendar import timegm
from pymongo import MongoClient, ASCENDING, DESCENDING
import pyvisa
import serial

# Setup and configure Mongo

In [4]:
def dbConnect():
    client = MongoClient('mongodb://localhost/')
    db = client.data
    collection = db.adr1datas
    db.collection_names()
    return collection
collection = dbConnect()

In [3]:
adr2infocollection = db.adr2infocollections
db.collection_names()

NameError: name 'db' is not defined

In [5]:
collection.full_name

'data.adr1datas'

# Add Single Data file to DB

### Returns dict with random data

In [17]:
def rand_data():
    post = {
        "timeStamp": timegm(datetime.now().timetuple()),
        "baseTemp": .05*np.random.rand()+.05,
        "threeKTemp": 0.7*np.random.rand()+2.8,
        "sixtyKTemp": np.random.rand()+60,
        "magnetVoltage": np.random.rand(),
        "psVoltage": np.random.rand(),
        "psCurrent": np.random.rand(),
        "currentJob": "None",
        "percentComplete": 0,
        "switchState": "Closed"
    }
    return post
rand_data()

{'baseTemp': 0.087650353245358,
 'currentJob': 'None',
 'magnetVoltage': 0.4452160265629367,
 'percentComplete': 0,
 'psCurrent': 0.9102815720770526,
 'psVoltage': 0.2778003531985641,
 'sixtyKTemp': 60.429288673929356,
 'switchState': 'Closed',
 'threeKTemp': 3.2061069052552913,
 'timeStamp': 1420547121}

In [14]:
for data in collection.find().limit(10):
    print(data)

{u'_id': ObjectId('54da8d2dc860b11ad48cee9b'), u'threeKTemp': u'+293.410E+0', u'timeStamp': 1423591132, u'oneKTemp': u'+250.412E+0', u'currentJob': u'None', u'switchState': u'Closed', u'psVoltage': u'+0.000000E+00\n', u'percentComplete': 0, u'psCurrent': u'+0.000000E+00\n', u'sixtyKTemp': u'+294.043E+0', u'magnetVoltage': u'-00.5258E-3', u'baseTemp': u'+00.000'}
{u'_id': ObjectId('54da8d2ec860b11ad48cee9c'), u'threeKTemp': u'+293.410E+0', u'timeStamp': 1423591134, u'oneKTemp': u'+250.838E+0', u'currentJob': u'None', u'switchState': u'Closed', u'psVoltage': u'+0.000000E+00\n', u'percentComplete': 0, u'psCurrent': u'+0.000000E+00\n', u'sixtyKTemp': u'+294.044E+0', u'magnetVoltage': u'-00.5236E-3', u'baseTemp': u'+00.000'}
{u'_id': ObjectId('54da8d2fc860b11ad48cee9d'), u'threeKTemp': u'+293.410E+0', u'timeStamp': 1423591135, u'oneKTemp': u'+250.599E+0', u'currentJob': u'None', u'switchState': u'Closed', u'psVoltage': u'+0.000000E+00\n', u'percentComplete': 0, u'psCurrent': u'+0.000000E+00

## Get Data from Instruments

### Create Instrument Objects

In [6]:
def connectInstruments():
    rm = pyvisa.ResourceManager()
    rm.list_resources()
    inst1 = rm.open_resource('GPIB0::1::INSTR')
    inst2 = rm.open_resource('GPIB0::2::INSTR')
    inst3 = rm.open_resource('GPIB0::6::INSTR')
    return [inst1,inst2,inst3]

In [7]:
x = connectInstruments()
x[0].ask('*IDN?')

'LSCI,MODEL340,341911,042304'

### Get Data Object

In [35]:
rm = pyvisa.ResourceManager()
ls1 = rm.open_resource('GPIB0::1::INSTR')

In [9]:
ls1.encoding

NameError: name 'ls1' is not defined

In [12]:
inst[0].encoding = 'utf-8'

In [15]:
inst = connectInstruments()
def get_data(inst):
    post = {
        "timeStamp": timegm(datetime.now().timetuple()),
        "baseTemp": float(inst[1].ask('KRDG? A')),
        "oneKTemp": float(inst[0].ask('KRDG? C')),
        "threeKTemp": float(inst[0].ask('KRDG? A')),
        "sixtyKTemp": float(inst[0].ask('KRDG? B')),
        "magnetVoltage": float(inst[0].ask('SRDG? D')),
        "psVoltage": 0,
        "psCurrent": 0,
        "currentJob": "None",
        "percentComplete": 0,
        "switchState": "Closed"
    }
    return post
get_data(inst)

{'baseTemp': 0.0,
 'currentJob': 'None',
 'magnetVoltage': -0.00836,
 'oneKTemp': 160.347,
 'percentComplete': 0,
 'psCurrent': 0,
 'psVoltage': 0,
 'sixtyKTemp': 70.939,
 'switchState': 'Closed',
 'threeKTemp': 146.515,
 'timeStamp': 1433541452}

###Validate Data, Determine Fridge Status

In [ ]:
def fridgeStatus(data, infocollection):
    

### Update every 5 seconds

In [ ]:
while True:
    try:
        temp = get_data(inst)
        
    except UnicodeDecodeError:
        print("Read error at " + datetime.now().strftime('%H:%M:%S'))
        inst = connectInstruments()
        
    else:
        if (temp['sixtyKTemp'] <= 290):
            try:
                collection.insert(temp)
            except:
                print("Database error at " + datetime.now().strftime('%H:%M:%S'))
                
    time.sleep(5)

Read error at 21:57:44
Read error at 21:58:15

In [ ]:
while True:
    temp = get_data()
    if (temp['sixtyKTemp'] <= 290):
            collection.insert(temp)
    time.sleep(5)

In [10]:
temp['sixtyKTemp']

296.278

'10:01:18'